In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

path = r
df = pd.read_excel(path)
df.head()

In [32]:
n = 57

In [33]:
five_mults = [x for x in range(5,105,5)]

if n > 38:
    for i in range(len(five_mults)):
        if n > five_mults[i] and n <= five_mults[i+1]:
            print(five_mults[i], five_mults[i+1])
            if five_mults[i+1] - n < 3:
                print(five_mults[i+1])
                n = i
            else:
                n = n
                print(n)
else:
    n = n
    print(n)

55 60
57
